# README

This notebook is organised as follows:

1.   **Recover** section contains recovery setup with an example of saving and loading data to google drive. This was used for longer experiments where the instance could timeout, such as LIME evaluation on Imagenet.
2.   **Setup** section contains commands for installing dependencies and defines functions to initialise datasets and models.
3.   **Metrics** section contains the implementations of the five metrics that we used to evaluate the explainability techniques.
4.   **Explainers** section contains wrappers/adapters for the different explainers to provide a unified interface to execute the experiments.
5.   **Experiments** section contains executions of all the experiments we conducted. Sections 2-4 should be expecuted prior to running anything in this section, and, optionally Section 1, as well as the Utils subsection, containing helper functions for visualization.
The comparative evaluation of the XAI techniques itself is organised by dataset, by model, then by method. Prior to running an experiment for a particular technique, initialisation for the dataset and model have to be executed.
Finally, this section also contains the code used to generate images included in the paper.

Known issues:

1.   GradCAM and GradCAM++ explainers seem to have a memory leak. This is not really perceptible in small-scale experiments, but running fidelity evaluation for 1000 Imagenet images using GradCAM or GradCAM++ explainers crashes the instance. A workaround for this is to re-initialise the model before running fidelity calculation.



# Recover

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# np.save("/content/drive/MyDrive/Uni/thesis/imagenet_densenet121_shap.npy", exps)
# exps = np.load("/content/drive/MyDrive/Uni/thesis/imagenet_densenet121_shap.npy", exps)

# Setup

In [ ]:
%%capture
%pip install shap grad-cam captum detectors tqdm
%pip install git+https://github.com/marcotcr/lime

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json
import itertools
import sklearn
import sklearn.cluster
from functools import partial
import random
from statistics import mean
import time
from tqdm import tqdm
from functools import partial

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

import detectors
import timm

from skimage.segmentation import slic, mark_boundaries, quickshift
from skimage.util import img_as_float32 as img_as_float
from skimage.util import random_noise

In [ ]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
# When running on the CuDNN backend, two further options must be set
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tqdm = partial(tqdm, position=0, leave=True)

## Utils

In [ ]:
def clean_axis(ax):
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.spines['bottom'].set_color(None)
    ax.spines['top'].set_color(None)
    ax.spines['right'].set_color(None)
    ax.spines['left'].set_color(None)

## Dataset

In [ ]:
N_IMAGES_PER_CLASS = 10

In [ ]:
imagenet_idx_to_imagenette = {
    0: 0,    # tench
    217: 1,  # English springer
    482: 2,  # cassette player
    491: 3,  # chain saw
    497: 4,  # church
    566: 5,  # French horn
    569: 6,  # garbage truck
    571: 7,  # gas pump
    574: 8,  # golf ball
    701: 9,  # parachute
}

In [ ]:
def get_sample_images(dataset, classes, n_images_per_class=N_IMAGES_PER_CLASS):
    images_by_class = [[] for c in classes]

    for image, cls in dataset:
        if len(images_by_class[cls]) == n_images_per_class:
            continue
        images_by_class[cls].append(image)
        if all(len(v) == n_images_per_class for v in images_by_class):
            break

    images = [j for i in images_by_class for j in i]
    labels = [c for c, i in enumerate(images_by_class) for j in i]
    return images, labels, images_by_class


def show_images_by_class(images_by_class):
    plt.figure()
    nrow = len(images_by_class)
    ncol = min(max([len(x) for x in images_by_class]), 10)
    _, axarr = plt.subplots(nrow, ncol, figsize=(ncol, nrow))
    for i, cls_images in enumerate(images_by_class):
        for j, img in enumerate(cls_images[:10]):
            axarr[i,j].imshow(img.convert('RGB'))
            axarr[i,j].axis("off")


def get_random_images_of_each_class(images_by_class, n_img=1):
    res = []
    for cls_images in images_by_class:
        res.extend(random.sample(cls_images, n_img))
    return res


In [ ]:
from torchvision.datasets import CIFAR100, CIFAR10, Imagenette, SVHN


def init_dataset(dataset_name):
    if dataset_name == "cifar100":
        dataset = CIFAR100(root="./data", download=True, train=False)
        classes = dataset.classes
        size, high_width, channels = 32, 32 * 32, 3
    elif dataset_name == "cifar10":
        dataset = CIFAR10(root="./data", download=True, train=False)
        classes = dataset.classes
        size, high_width, channels = 32, 32 * 32, 3
    elif dataset_name == "imagenet":
        dataset = Imagenette(
            root="./data",
            # download=True,
            split="val",
            transform=transforms.Compose(
                [
                    transforms.Resize((256, 256)),
                    transforms.CenterCrop(224)
                ]
            )
        )
        classes = dataset.classes
        size, high_width, channels = 224, 224 * 224, 3
    elif dataset_name == "svhn":
        dataset = SVHN(root="./data", download=True, split="test")
        classes = list(range(10))
        size, high_width, channels = 32, 32 * 32, 3
    else:
        raise Exception("Unsupported dataset")

    return dataset, classes, size, high_width, channels


## Model

In [ ]:
from detectors.data import CIFAR100_DEFAULT_MEAN, CIFAR100_DEFAULT_STD
from detectors.data import CIFAR10_DEFAULT_MEAN, CIFAR10_DEFAULT_STD
from detectors.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from detectors.data import SVHN_DEFAULT_MEAN, SVHN_DEFAULT_STD


def get_preprocess_transform(dataset_name):
    if dataset_name == "cifar100":
        mean, std = CIFAR100_DEFAULT_MEAN, CIFAR100_DEFAULT_STD
    elif dataset_name == "cifar10":
        mean, std = CIFAR10_DEFAULT_MEAN, CIFAR10_DEFAULT_STD
    elif dataset_name == "imagenet":
        mean, std = IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
    elif dataset_name == "svhn":
        mean, std = SVHN_DEFAULT_MEAN, SVHN_DEFAULT_STD
    else:
        raise Exception("Unsupported dataset")

    return transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ]
    )


def get_model(architecture_name, dataset_name):
    if architecture_name not in ("vgg16_bn", "resnet50", "densenet121"):
        raise Exception("Unsupported architecture")
    if dataset_name not in ("cifar100", "cifar10", "imagenet", "svhn"):
        raise Exception("Unsupported dataset")

    if dataset_name == "imagenet":
        weights = {
            "vgg16_bn": models.VGG16_BN_Weights,
            "resnet50": models.ResNet50_Weights,
            "densenet121": models.DenseNet121_Weights,
        }[architecture_name]
        model = getattr(models, architecture_name)(weights=weights)
    else:
        model = timm.create_model(f"{architecture_name}_{dataset_name}", pretrained=True)

    if architecture_name == "resnet50":
        target_layers = [model.layer4[-1]]
    elif architecture_name == "vgg16_bn":
        target_layers = [model.features[-2]]
    elif architecture_name == "densenet121":
        target_layers = [model.features[-1]]
    return model, target_layers


def get_batch_predict(model, preprocess_transform):

    def batch_predict(images):
        model.eval()
        batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)

        model.to(device)
        batch = batch.to(device)

        logits = model(batch)
        probs = F.softmax(logits, dim=1)
        return probs.detach().cpu().numpy()

    return batch_predict

# **Metrics**

In [ ]:
# UTILS

def dist(img1, img2):
    return np.sqrt(np.sum((img2 - img1)**2))


def auc(arr):
    """Returns normalized Area Under Curve of the array."""
    return (arr.sum() - arr[0] / 2 - arr[-1] / 2) / (arr.shape[0] - 1)


def minmax_scale(img):
    if np.min(img) == np.max(img):
        return img
    return (img - np.min(img)) / (np.max(img) - np.min(img))

In [ ]:
def calculate_identity(exps1, exps2):
    dis = np.array([np.array_equal(exps1[i],exps2[i]) for i in range(len(exps1))])
    total = dis.shape[0]
    true = np.sum(dis)
    score = (total-true)/total
    print('true:', true, 'wrong:', total-true, 'total:', total)
    return score*100, true, total


def calculate_separability(exps):
    wrong = 0
    for i in range(exps.shape[0]):
        for j in range(exps.shape[0]):
            if i == j:
                continue
            eq = np.array_equal(exps[i],exps[j])
            if eq:
                wrong += 1
    total = exps.shape[0]
    score = 100*abs(wrong)/(total**2-total)
    print('true:', total**2-total-wrong, 'wrong:', wrong, 'total:', total**2-total)
    return wrong, total, total**2-total, score


def calculate_fidelity(images, explanations, model, step, print_every=1, get_class=None):
    n_steps = (HW + step - 1) // step

    print_steps = [int((n_steps+1) * p) for p in (0, 0.25, 0.5, 0.75)]
    print_img_size = 4  #inches
    print_img_count = len(print_steps) + 2
    print_inds = list(range(0, len(images), print_every)) if print_every else []

    scores = []

    for pind, (image, explanation) in tqdm(enumerate(zip(images, explanations))):
        img = img_as_float(image)
        img_scores = np.empty(n_steps + 1)
        salient_order = np.flip(np.argsort(explanation.reshape(-1, HW), axis=1), axis=-1)
        pred = torch.tensor(model(img))
        _, c = torch.max(pred, 1)
        c = c[0]


        if pind in print_inds:
            plt.figure(figsize=(print_img_size * print_img_count, print_img_size))

            plt.subplot(1, print_img_count, 1)
            plt.title("Explanation", fontsize=14)
            plt.imshow(explanation)
            if not get_class:
                plt.axis('off')
            else:
                ax = plt.gca()
                clean_axis(ax)
                ax.set_ylabel(get_class(c), fontsize=14)

        for i in range(n_steps+1):
            pred = torch.tensor(model(img))
            # pr, cl = torch.topk(pred, 2)
            img_scores[i] = pred[0, c]

            if pind in print_inds and i in print_steps:
                plt.subplot(1, print_img_count, print_steps.index(i) + 2)
                plt.title('Pixels deleted {:.1f}%, P={:.4f}'.format(100 * i / n_steps, img_scores[i]), fontsize=14)
                plt.axis('off')
                plt.imshow(img)

            if i < n_steps:
                # img = img.reshape(1, HW, CH)
                # coords = salient_order[:, step * i:step * (i + 1)]
                # img[:,coords,:] = 0
                # img = img.reshape(SIZE, SIZE, CH)
                coords = salient_order[:, step * i:step * (i + 1)]
                img.reshape(1, HW, CH)[:,coords,:] = 0

        score = auc(img_scores)

        if pind in print_inds:
            plt.subplot(1, print_img_count, print_img_count)
            plt.xlim(-0.1, 1.1)
            plt.ylim(0, 1.05)
            plt.fill_between(np.arange(n_steps+1) / n_steps, 0, img_scores, alpha=0.4)
            plt.title(f"Score: {round(score, 4)}", fontsize=14)
            plt.xlabel("Pixels deleted", fontsize=14)
            plt.ylabel("Prediction score", fontsize=14)

        scores.append(score)

    return scores


def calculate_stability(images, explanations, n=5, print_every=1):
    num_images = len(images)
    images_as_float = [img_as_float(img) for img in images]
    distances = np.zeros((num_images, num_images))
    for i in range(num_images):
        for j in range(i, num_images):
            distances[i,j] = dist(images_as_float[i], images_as_float[j])
            distances[j,i] = distances[i,j]

    print_img_size = 2  #inches
    print_img_count = n * 2 + 2
    print_inds = list(range(0, len(images), print_every)) if print_every else []

    score = []
    for i, (image, image_float, explanation) in tqdm(enumerate(zip(images, images_as_float, explanations))):
        top_n_similar_images_idx = np.argsort(distances[i])[1:n+1]
        top_n_similar_images = [images[ix] for ix in top_n_similar_images_idx]
        top_n_similar_images_float = [images_as_float[ix] for ix in top_n_similar_images_idx]
        top_n_similar_explanations = [explanations[ix] for ix in top_n_similar_images_idx]

        if i in print_inds:
            print(i)
            plt.figure(figsize=(print_img_size * print_img_count, print_img_size))

            plt.subplot(1, print_img_count, 1)
            plt.axis('off')
            # plt.title('Original image')
            plt.imshow(image)
            plt.subplot(1, print_img_count, 2)
            # plt.title('Original explanation')
            plt.axis('off')
            plt.imshow(explanation)

        l_score = []
        for j, (s_i, s_i_f, s_e) in enumerate(zip(top_n_similar_images, top_n_similar_images_float, top_n_similar_explanations)):
            if i in print_inds:
                plt.subplot(1, print_img_count, 3+j*2)
                plt.axis('off')
                plt.imshow(s_i)
                plt.subplot(1, print_img_count, 4+j*2)
                plt.axis('off')
                plt.imshow(s_e)

            exp_diff = explanation - s_e
            exp_diff_norm = np.linalg.norm(exp_diff)
            im_diff = image_float - s_i_f
            im_diff_norm = np.linalg.norm(im_diff)
            s = exp_diff_norm / im_diff_norm
            l_score.append(s)

        score.append(max(l_score))

    return score


def calculate_stability_v2(images, explanations, explain_func):
    noise_technique = ("gaussian", "poisson", "speckle", "s&p")
    noise_technique = ("gaussian", "gaussian", "gaussian", "gaussian", "gaussian")
    n = len(noise_technique)
    score = []
    for image, explanation in zip(images, explanations):
        image = img_as_float(image)
        noisy_images = [random_noise(image, nt).astype(np.float32) for nt in noise_technique]
        noisy_explanations = [explain_func(ni) for ni in noisy_images]

        plt.figure(figsize=(15, 15))

        plt.subplot(1, 2 + 2*n, 1)
        plt.axis('off')
        plt.imshow(image)
        plt.subplot(1, 2 + 2*n, 2)
        plt.axis('off')
        plt.imshow(explanation)

        l_score = []
        for j, (s_i, s_e) in enumerate(zip(noisy_images, noisy_explanations)):
            plt.subplot(1, 2 + 2*n, 3+j*2)
            plt.axis('off')
            plt.imshow(s_i)
            plt.subplot(1, 2 + 2*n, 4+j*2)
            plt.axis('off')
            plt.imshow(s_e)

            exp_diff = explanation - s_e
            exp_diff_norm = np.linalg.norm(exp_diff)
            im_diff = img_as_float(image) - img_as_float(s_i)
            im_diff_norm = np.linalg.norm(im_diff)
            s = exp_diff_norm / im_diff_norm
            l_score.append(s)

        score.append(max(l_score))

    return score

# Explainers

In [ ]:
class ExplainerAdapter(object):
    def __init__(self, model, target_layers, preprocess_transform, img_size, img_channels):
        self.model = model
        self.target_layers = target_layers
        self.preprocess_transform = preprocess_transform
        self.batch_predict = get_batch_predict(model, preprocess_transform)

        self.img_size = img_size
        self.img_channels = img_channels

        self.explainer = self.init_explainer()

    def init_explainer(self):
        raise NotImplementedError

    def pre_explain(self):
        pass

    def get_extra_from_image(self, image):
        return {}

    def get_input_from_image(self, image):
        return image

    def explain(self, input, **kwargs):
        raise NotImplementedError

    def postprocess_explanation(self, explanation):
        raise NotImplementedError

    def timed_single_explain(self, image):
        self.pre_explain()

        input = self.get_input_from_image(image)
        extra = self.get_extra_from_image(image)

        t = time.time()
        exp = self.explain(input, **extra)
        t = time.time() - t

        return self.postprocess_explanation(exp), t

    def timed_batch_explain(self, images):
        exps = []
        times = []

        for image in tqdm(images):
            exp, t = self.timed_single_explain(image)
            exps.append(exp)
            times.append(t)

        return np.array(exps), np.mean(times)

### LIME

In [ ]:
from lime.lime_image import LimeImageExplainer
from lime.wrappers.scikit_image import SegmentationAlgorithm

In [ ]:
class LimeExplainerAdapter(ExplainerAdapter):
    def init_explainer(self):
        return LimeImageExplainer()

    def get_input_from_image(self, image):
        return np.array(image)

    def explain(self, input):
        return self.explainer.explain_instance(
            input,
            self.batch_predict, # classification function
            top_labels=5,
            # hide_color=0,
            num_samples=1000,
            # segmentation_fn=SegmentationAlgorithm(
            #     'quickshift', kernel_size=1, max_dist=16, ratio=0.2, random_seed=1
            # )
            progress_bar=False,
        )

    def postprocess_explanation(self, exp):
        exp = exp.get_image_and_mask(
            exp.top_labels[0],
            positive_only=True,
            num_features=5,
            hide_rest=True
        )[1]
        exp = minmax_scale(exp)
        return exp

#### Old

In [ ]:
# def bulk_exp(explainer, images):
#     return [
#         explainer.explain_instance(
#           np.array(image),
#           batch_predict, # classification function
#           top_labels=5,
#           hide_color=0,
#           num_samples=1000,
#           segmentation_fn=SegmentationAlgorithm('quickshift', kernel_size=1, max_dist=16, ratio=0.2, random_seed=1)
#         )
#         for image in images
#     ]


# def get_imgs_from_exps(exps):
#     return np.array(
#         [
#             exp.get_image_and_mask(
#                 exp.top_labels[0],
#                 positive_only=True,
#                 num_features=5,
#                 hide_rest=True
#             )[1]
#             for exp in exps
#         ]
#     )
#

# explainer = LimeImageExplainer()

# exps = get_imgs_from_exps(bulk_exp(explainer, sample_images))

### SHAP

In [ ]:
import shap

In [ ]:
class ShapExplainerAdapter(ExplainerAdapter):
    def init_explainer(self):
        return shap.Explainer(
            self.batch_predict,
            shap.maskers.Image(
                f"blur({self.img_size},{self.img_size})",
                (self.img_size, self.img_size, self.img_channels)
            )
        )

    def get_input_from_image(self, image):
        return np.array([image])

    def explain(self, input):
        return self.explainer(input, outputs=shap.Explanation.argsort.flip[:1], silent=True)

    def postprocess_explanation(self, exp):
        exp = exp[0][:,:,:,0].values
        exp = np.mean(exp, axis=2)
        exp[exp < 0] = 0
        exp = minmax_scale(exp)
        return exp

#### Old

In [ ]:
# def bulk_exp(explainer, images):
#     return explainer(
#         np.array(images),
#         outputs=shap.Explanation.argsort.flip[:1],
#     )


# def get_imgs_from_exps(exps):
#     def norm(arr):
#         return np.mean((arr - np.min(arr)) / (np.max(arr) - np.min(arr)), axis=2)

#     return np.array([norm(x[:,:,:,0].values) for x in exps[:]])

# explainer = shap.Explainer(
#     batch_predict,
#     shap.maskers.Image(f"blur({SIZE},{SIZE})", (SIZE, SIZE, CH))
# )

# exps = get_imgs_from_exps(bulk_exp(explainer, imgs))


### GradCam/GradCam++

In [ ]:
from pytorch_grad_cam import GradCAMPlusPlus, GradCAM, FullGrad, EigenCAM, EigenGradCAM, AblationCAM, ScoreCAM, HiResCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
class GradExplainerAdapter(ExplainerAdapter):
    def pre_explain(self):
        self.model.eval()
        self.model.to(device)

    def get_input_from_image(self, image):
        input = self.preprocess_transform(image).unsqueeze(0).to(device)
        input.requires_grad = True
        return input

    def postprocess_explanation(self, exp):
        return exp[0]

class CAMExplainerAdapter(GradExplainerAdapter):
    def explain(self, input):
        return self.explainer(input_tensor=input)


class GradCAMExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return GradCAM(model=self.model, target_layers=self.target_layers)


class GradCAMPlusPlusExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return GradCAMPlusPlus(model=self.model, target_layers=self.target_layers)


class AblationCAMExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return AblationCAM(model=self.model, target_layers=self.target_layers)


class ScoreCAMExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return ScoreCAM(model=self.model, target_layers=self.target_layers)


class EigenCAMExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return EigenCAM(model=self.model, target_layers=self.target_layers)


class EigenGradCAMExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return EigenGradCAM(model=self.model, target_layers=self.target_layers)


class FullGradExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return FullGrad(model=self.model, target_layers=self.target_layers)


class HiResCAMExplainerAdapter(CAMExplainerAdapter):
    def init_explainer(self):
        return HiResCAM(model=self.model, target_layers=self.target_layers)


#### Old

In [ ]:
# def bulk_exp(explainer, images):
#     return explainer(
#         input_tensor=torch.stack(
#             tuple(preprocess_transform(i) for i in images),
#             dim=0,
#         ),
#     )


# def get_imgs_from_exps(exps):
#     return exps

# explainer = GradCAM(model=self.model, target_layers=self.model.layer4[-1:])

# exps = get_imgs_from_exps(bulk_exp(explainer, imgs))


### Intgrad, SmoothGrad

In [ ]:
from captum.attr import IntegratedGradients, NoiseTunnel

In [ ]:
class IntGradExplainerAdapter(GradExplainerAdapter):
    def pre_explain(self):
        super().pre_explain()
        # self.model.zero_grad()

    def init_explainer(self):
        return IntegratedGradients(self.model)

    def get_extra_from_image(self, image):
        return {"target": int(np.argmax(self.batch_predict([image]), axis=1)[0])}

    def postprocess_explanation(self, exp):
        exp = exp.squeeze(0).cpu().detach().numpy()
        exp = np.transpose(exp, (1, 2, 0))
        exp = np.mean(exp, axis=2)
        exp[exp < 0] = 0
        exp = minmax_scale(exp)
        return exp

    def explain(self, input, **kwargs):
        return self.explainer.attribute(
            inputs=input,
            baselines=input * 0,
            target=kwargs["target"],
            return_convergence_delta=False,
        )


class SmoothGradExplainerAdapter(IntGradExplainerAdapter):
    def init_explainer(self):
        return NoiseTunnel(super().init_explainer())

    def explain(self, input, **kwargs):
        return self.explainer.attribute(
            inputs=input,
            nt_type='smoothgrad',
            stdevs=0.2,
            nt_samples_batch_size=1,
            baselines=input * 0,
            target=kwargs["target"],
            return_convergence_delta=False,
            # internal_batch_size=1,
        )

#### Old

In [ ]:
# def bulk_exp(explainer, images, predictions):
#     # process one image at a time to avoid running out of memory
#     exps = []
#     model.zero_grad()
#     for image, prediction in zip(images, predictions):
#         input = preprocess_transform(image).unsqueeze(0).to(device)
#         input.requires_grad = True
#         exps.append(
#             explainer.attribute(
#                 inputs=input,
#                 nt_type='smoothgrad',
#                 stdevs=0.2,
#                 nt_samples_batch_size=1,
#                 baselines=input * 0,
#                 target=int(prediction),
#                 return_convergence_delta=False,
#                 internal_batch_size=1,
#             ).squeeze(0).cpu().detach().numpy()
#         )
#     return np.array(exps)


# def get_imgs_from_exps(exps):
#     def norm(arr):
#         return np.mean((arr - np.min(arr)) / (np.max(arr) - np.min(arr)), axis=2)

#     exps = np.transpose(exps, (0, 2, 3, 1))
#     return np.array([norm(x) for x in exps])


# model.to(device)
# model.eval()
# explainer = NoiseTunnel(IntegratedGradients(model))


# exps = get_imgs_from_exps(bulk_exp(explainer, imgs, preds))

# Experiments

## Utils

In [ ]:
# def evaluate_explainer(expr_name):
#     expr_class = {
#         "LIME": LimeExplainerAdapter,
#         "SHAP": ShapExplainerAdapter,
#         "GradCAM": GradCAMExplainerAdapter,
#         "GradCAM++": GradCAMPlusPlusExplainerAdapter,
#         "IntGrad": IntGradExplainerAdapter,
#         "SmoothGrad": SmoothGradExplainerAdapter,
#     }[expr_name]
#     print(f"Starting for {expr_name}, {model.pretrained_cfg['architecture']}")

#     expr = expr_class(model, target_layers, preprocess_transform, SIZE, CH)

#     exps, t = expr.timed_batch_explain(imgs)
#     exps_, t = expr.timed_batch_explain(imgs)
#     print(f"Mean time for {len(imgs)} explanations: {t}")

#     nrow, ncol = 2, 10
#     plt.figure()
#     _, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
#     for ix, i in enumerate(range(0, len(imgs), print_every_n)):
#         axarr[0, ix].imshow(imgs[i])
#         axarr[0, ix].axis(False)
#         axarr[1, ix].imshow(exps[i])
#         axarr[1, ix].axis(False)

#     scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
#     print(f"Fidelity score: {np.mean(scores)}")

#     scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
#     print(f"Stability score: {np.mean(scores)}")

#     score, _, _ = calculate_identity(exps, exps_)
#     print(f"Identity score: {score}")

#     _, _, _, score = calculate_separability(exps)
#     print(f"Separability score: {score}")

In [ ]:
def show_imgs_exps(imgs, exps, print_every_n):
    nrow, ncol = 2, 10
    plt.figure()
    _, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
    for ix, i in enumerate(range(0, len(imgs), print_every_n)):
        axarr[0, ix].imshow(imgs[i])
        axarr[0, ix].axis(False)
        axarr[1, ix].imshow(exps[i])
        axarr[1, ix].axis(False)

## CIFAR10

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset("cifar10")

In [ ]:
sample_images, sample_labels, sample_images_by_class = get_sample_images(dataset, classes, n_images_per_class=100)
imgs = sample_images
print_every_n = int(len(imgs) / 10)

### VGG16_BN

In [ ]:
model, target_layers = get_model("vgg16_bn", "cifar10")
preprocess_transform = get_preprocess_transform("cifar10")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 2, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix, i in enumerate(range(100, 110)):
    axarr[0, ix].imshow(exps[i])
    axarr[0, ix].axis(False)
    axarr[1, ix].imshow(exps_[i])
    axarr[1, ix].axis(False)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, [model.features[-2]], preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

### Resnet50

In [ ]:
model, target_layers = get_model("resnet50", "cifar10")
preprocess_transform = get_preprocess_transform("cifar10")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
%%capture
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
%%capture
exps_, t_ = expr.timed_batch_explain(imgs)

In [ ]:
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

### Densenet121

In [ ]:
model, target_layers = get_model("densenet121", "cifar10")
preprocess_transform = get_preprocess_transform("cifar10")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
%%capture
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
%%capture
exps_, t_ = expr.timed_batch_explain(imgs)

In [ ]:
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

## SVHN

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset("svhn")

In [ ]:
sample_images, sample_labels, sample_images_by_class = get_sample_images(dataset, classes, n_images_per_class=100)
imgs = sample_images
print_every_n = int(len(imgs) / 10)

### VGG16_BN

In [ ]:
model, target_layers = get_model("vgg16_bn", "svhn")
preprocess_transform = get_preprocess_transform("svhn")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 2, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix, i in enumerate(range(100, 110)):
    axarr[0, ix].imshow(exps[i])
    axarr[0, ix].axis(False)
    axarr[1, ix].imshow(exps_[i])
    axarr[1, ix].axis(False)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, [model.features[-2]], preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

### Resnet50

In [ ]:
model, target_layers = get_model("resnet50", "svhn")
preprocess_transform = get_preprocess_transform("svhn")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

### Densenet121

In [ ]:
model, target_layers = get_model("densenet121", "svhn")
preprocess_transform = get_preprocess_transform("svhn")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

## Imagenet

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset("imagenet")

In [ ]:
sample_images, sample_labels, sample_images_by_class = get_sample_images(dataset, classes, n_images_per_class=100)
imgs = sample_images
print_every_n = int(len(imgs) / 10)

### VGG16_BN

In [ ]:
model, target_layers = get_model("vgg16_bn", "imagenet")
preprocess_transform = get_preprocess_transform("imagenet")
batch_predict = get_batch_predict(model, preprocess_transform)


#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps1, t = expr.timed_batch_explain(imgs[:250])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime1.npy", exps1)

In [ ]:
exps1 = np.load("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime1.npy")

In [ ]:
exps2, t = expr.timed_batch_explain(imgs[250:500])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime2.npy", exps2)

In [ ]:
exps2 = np.load("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime2.npy")

In [ ]:
exps3, t = expr.timed_batch_explain(imgs[500:750])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime3.npy", exps3)

In [ ]:
exps3 = np.load("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime3.npy")

In [ ]:
exps4, t = expr.timed_batch_explain(imgs[750:])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime4.npy", exps4)

In [ ]:
exps4 = np.load("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime4.npy")

In [ ]:
exps = np.concatenate((exps1, exps2, exps3, exps4))

In [ ]:
exps.shape

In [ ]:
exps1_, t = expr.timed_batch_explain(imgs[:250])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime1x.npy", exps1_)

In [ ]:
exps1_ = np.load("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime1x.npy")

In [ ]:
exps2_, t = expr.timed_batch_explain(imgs[250:500])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime2x.npy", exps2_)

In [ ]:
exps3_, t = expr.timed_batch_explain(imgs[500:750])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime3x.npy", exps3_)

In [ ]:
exps4_, t = expr.timed_batch_explain(imgs[750:])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_vgg16bn_lime4x.npy", exps4_)

In [ ]:
exps_ = np.concatenate((exps1_, exps2_, exps3_, exps4_))

In [ ]:
exps.shape, exps_.shape

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 3, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix in range(ncol-1):
    for i in range(ix * print_every_n, (ix+1) * print_every_n):
        if not np.array_equal(exps[i], exps_[i]):
            axarr[0, ix].imshow(imgs[i])
            axarr[0, ix].axis(False)
            axarr[1, ix].imshow(exps[i])
            axarr[1, ix].axis(False)
            axarr[2, ix].imshow(exps_[i])
            axarr[2, ix].axis(False)
            break

In [ ]:
distances = [dist(e, e_) for e, e_ in zip(exps, exps_)]
np.mean(distances), np.std(distances)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
torch.cuda.empty_cache()

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 3, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix, i in enumerate(range(0, len(imgs), print_every_n)):
    axarr[0, ix].imshow(imgs[i])
    axarr[0, ix].axis(False)
    axarr[1, ix].imshow(exps[i])
    axarr[1, ix].axis(False)
    axarr[2, ix].imshow(exps_[i])
    axarr[2, ix].axis(False)

In [ ]:
distances = [dist(e, e_) for e, e_ in zip(exps, exps_)]
np.mean(distances), np.std(distances)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

### Resnet50

In [ ]:
model, target_layers = get_model("resnet50", "imagenet")
preprocess_transform = get_preprocess_transform("imagenet")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 3, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix in range(ncol-1):
    for i in range(ix * print_every_n, (ix+1) * print_every_n):
        if not np.array_equal(exps[i], exps_[i]):
            axarr[0, ix].imshow(imgs[i])
            axarr[0, ix].axis(False)
            axarr[1, ix].imshow(exps[i])
            axarr[1, ix].axis(False)
            axarr[2, ix].imshow(exps_[i])
            axarr[2, ix].axis(False)
            break

In [ ]:
distances = [dist(e, e_) for e, e_ in zip(exps, exps_)]
np.mean(distances), np.std(distances)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
torch.cuda.empty_cache()

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 3, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix, i in enumerate(range(0, len(imgs), print_every_n)):
    axarr[0, ix].imshow(imgs[i])
    axarr[0, ix].axis(False)
    axarr[1, ix].imshow(exps[i])
    axarr[1, ix].axis(False)
    axarr[2, ix].imshow(exps_[i])
    axarr[2, ix].axis(False)

In [ ]:
distances = [dist(e, e_) for e, e_ in zip(exps, exps_)]
np.mean(distances), np.std(distances)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

### Densenet121

In [ ]:
model, target_layers = get_model("densenet121", "imagenet")
preprocess_transform = get_preprocess_transform("imagenet")
batch_predict = get_batch_predict(model, preprocess_transform)

#### LIME

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps1, t = expr.timed_batch_explain(imgs[:500])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_densenet121_lime1.npy", exps1)

In [ ]:
exps2, t = expr.timed_batch_explain(imgs[500:])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_densenet121_lime2.npy", exps2)

In [ ]:
exps = np.concatenate((exps1, exps2))

In [ ]:
exps1_, t = expr.timed_batch_explain(imgs[:500])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_densenet121_lime1x.npy", exps1_)

In [ ]:
exps2_, t = expr.timed_batch_explain(imgs[500:])
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
np.save("/content/drive/MyDrive/Uni/thesis/imagenet_densenet121_lime2x.npy", exps2_)

In [ ]:
exps_ = np.concatenate((exps1_, exps2_))

In [ ]:
exps.shape, exps_.shape

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
show_imgs_exps(imgs, exps_, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 3, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix, i in enumerate(range(0, len(imgs), print_every_n)):
    axarr[0, ix].imshow(imgs[i])
    axarr[0, ix].axis(False)
    axarr[1, ix].imshow(exps[i])
    axarr[1, ix].axis(False)
    axarr[2, ix].imshow(exps_[i])
    axarr[2, ix].axis(False)

In [ ]:
distances = [dist(e, e_) for e, e_ in zip(exps, exps_)]
np.mean(distances), np.std(distances)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SHAP

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### GradCAM++

In [ ]:
expr = GradCAMPlusPlusExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### IntGrad

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

#### SmoothGrad

In [ ]:
torch.cuda.empty_cache()

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
exps_, t_ = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t_}")

In [ ]:
show_imgs_exps(imgs, exps, print_every_n)

In [ ]:
scores = calculate_fidelity(imgs, exps, lambda x: expr.batch_predict([x]), step=SIZE, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
scores = calculate_stability(imgs, exps, n=5, print_every=print_every_n)
np.mean(scores), np.std(scores)

In [ ]:
score, true, total = calculate_identity(exps, exps_)
score

In [ ]:
nrow, ncol = 3, 10
plt.figure()
_, axarr = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for ix, i in enumerate(range(0, len(imgs), print_every_n)):
    axarr[0, ix].imshow(imgs[i])
    axarr[0, ix].axis(False)
    axarr[1, ix].imshow(exps[i])
    axarr[1, ix].axis(False)
    axarr[2, ix].imshow(exps_[i])
    axarr[2, ix].axis(False)

In [ ]:
distances = [dist(e, e_) for e, e_ in zip(exps, exps_)]
np.mean(distances), np.std(distances)

In [ ]:
wrong, total, evaluated, score = calculate_separability(exps)
score

## Explanations Example

In [ ]:
ds_names = ["cifar10", "svhn", "imagenet"]
imgs_by_ds = {}
classes_by_ds = {}
models_by_ds = {}
settings_by_ds = {}
ixs = (3,5,8)
n_imgs_per_ds = len(ixs)

for ds_name in ds_names:
    print(ds_name)
    dataset, classes, SIZE, HW, CH = init_dataset(ds_name)
    sample_images, _, _ = get_sample_images(dataset, classes, n_images_per_class=1)

    imgs_by_ds[ds_name] = [sample_images[ix] for ix in ixs]
    classes_by_ds[ds_name] = classes

    settings_by_ds[ds_name] = (SIZE, HW, CH)

    model, target_layers = get_model("resnet50", ds_name)
    preprocess_transform = get_preprocess_transform(ds_name)
    batch_predict = get_batch_predict(model, preprocess_transform)
    models_by_ds[ds_name] = (model, target_layers, preprocess_transform, batch_predict)

In [ ]:
for ds_name in ds_names:
    model, target_layers = get_model("resnet50", ds_name)
    preprocess_transform = get_preprocess_transform(ds_name)
    batch_predict = get_batch_predict(model, preprocess_transform)
    models_by_ds[ds_name] = (model, target_layers, preprocess_transform, batch_predict)

In [ ]:
explainers = {
    "LIME": LimeExplainerAdapter,
    "SHAP": ShapExplainerAdapter,
    "GradCAM": GradCAMExplainerAdapter,
    "GradCAM++": GradCAMPlusPlusExplainerAdapter,
    "IntGrad": IntGradExplainerAdapter,
    "SmoothGrad": SmoothGradExplainerAdapter,
}

In [ ]:
nrow, ncol = len(ixs) * len(imgs_by_ds), 1 + len(explainers)
plt.figure()
imsize = 2
_, axarr = plt.subplots(nrow, ncol, figsize=(imsize*ncol, imsize*nrow))

for i, (ds_name, imgs) in enumerate(imgs_by_ds.items()):
    model, target_layers, preprocess_transform, batch_predict = models_by_ds[ds_name]
    SIZE, HW, CH = settings_by_ds[ds_name]

    preds = [np.argmax(p) for p in batch_predict(imgs)]
    preds = [
        classes_by_ds[ds_name][p]
        if ds_name != "imagenet"
        else classes_by_ds[ds_name][imagenet_idx_to_imagenette[p]][0]
        for p in preds
    ]
    print(preds)
    for k, (img, pred) in enumerate(zip(imgs, preds)):
        ax = axarr[i * n_imgs_per_ds + k, 0]
        ax.imshow(img)
        if i * n_imgs_per_ds + k == 0:
            ax.set_title("Image")

        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        ax.xaxis.set_ticks([])
        ax.yaxis.set_ticks([])
        ax.spines['bottom'].set_color(None)
        ax.spines['top'].set_color(None)
        ax.spines['right'].set_color(None)
        ax.spines['left'].set_color(None)
        ax.set_ylabel(pred, fontsize=14)


    for j, (expr_name, expr_class) in enumerate(explainers.items(), start=1):
        expr = expr_class(model, target_layers, preprocess_transform, SIZE, CH)

        exps, _ = expr.timed_batch_explain(imgs)
        for k, (img, exp) in enumerate(zip(imgs, exps)):
            axarr[i * n_imgs_per_ds + k, j].imshow(exp)
            axarr[i * n_imgs_per_ds + k, j].axis("off")
            if (i * n_imgs_per_ds + k) == 0:
                axarr[ i * n_imgs_per_ds + k, j].set_title(expr_name, fontsize=14)

## Stability example

## Fidelity example

In [ ]:
ds_name = "imagenet"

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset(ds_name)

In [ ]:
sample_images, sample_labels, sample_images_by_class = get_sample_images(dataset, classes, n_images_per_class=10)

In [ ]:
model, target_layers = get_model("resnet50", ds_name)
preprocess_transform = get_preprocess_transform(ds_name)
batch_predict = get_batch_predict(model, preprocess_transform)

In [ ]:
import json
with open("/content/imagenet_class_index.json", "r") as f:
    imagenet_classes = json.load(f)

In [ ]:
imgs = [sample_images[3]]
pfunc = lambda x: expr.batch_predict([x])
get_class = lambda x: imagenet_classes[int(x)]

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
calculate_fidelity(imgs, exps, pfunc, step=SIZE, print_every=1, get_class=get_class)

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
calculate_fidelity(imgs, exps, pfunc, step=SIZE, print_every=1, get_class=get_class)

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
calculate_fidelity(imgs, exps, pfunc, step=SIZE, print_every=1, get_class=get_class)

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
calculate_fidelity(imgs, exps, pfunc, step=SIZE, print_every=1, get_class=get_class)

In [ ]:
ds_name = "svhn"

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset(ds_name)

In [ ]:
sample_images, sample_labels, sample_images_by_class = get_sample_images(dataset, classes, n_images_per_class=10)

In [ ]:
model, target_layers = get_model("resnet50", ds_name)
preprocess_transform = get_preprocess_transform(ds_name)
batch_predict = get_batch_predict(model, preprocess_transform)

In [ ]:
sample_images[52]

In [ ]:
imgs = [sample_images[51], sample_images[52]]
pfunc = lambda x: expr.batch_predict([x])
get_class = lambda x: classes[int(x)]

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
calculate_fidelity(imgs, exps, pfunc, step=SIZE, print_every=1, get_class=get_class)

In [ ]:
expr = SmoothGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)
exps, t = expr.timed_batch_explain(imgs)

In [ ]:
calculate_fidelity(imgs, exps, pfunc, step=SIZE, print_every=1, get_class=get_class)

## Identity example

In [ ]:
ds_name = "imagenet"

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset(ds_name)

In [ ]:
sample_images, sample_labels, sample_images_by_class = get_sample_images(dataset, classes, n_images_per_class=10)

In [ ]:
sample_images[66]

In [ ]:
imgs = [sample_images[11], sample_images[42], sample_images[66]]
n_imgs = len(imgs)

In [ ]:
model, target_layers = get_model("resnet50", ds_name)
preprocess_transform = get_preprocess_transform(ds_name)
batch_predict = get_batch_predict(model, preprocess_transform)

In [ ]:
explainers = {
    "LIME": LimeExplainerAdapter,
    "SmoothGrad": SmoothGradExplainerAdapter,
}

In [ ]:
preds = [np.argmax(p) for p in batch_predict(imgs)]
preds = [
    classes[p]
    if ds_name != "imagenet"
    else classes[imagenet_idx_to_imagenette[p]][0]
    for p in preds
]
print(preds)

In [ ]:
nrow, ncol = 1 + len(explainers), n_imgs * 2
plt.figure()
imsize = 2
_, axarr = plt.subplots(nrow, ncol, figsize=(imsize*ncol, imsize*nrow))

for k, (img, pred) in enumerate(zip(imgs, preds)):
    for i in range(2):
        ax = axarr[0, k * 2 + i]
        ax.imshow(img)
        # ax.axis("off")
        if k * 2 + i == 0:
            ax.set_ylabel("Image", fontsize=14)

        ax.set_title(pred)
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        ax.xaxis.set_ticks([])
        ax.yaxis.set_ticks([])
        ax.spines['bottom'].set_color(None)
        ax.spines['top'].set_color(None)
        ax.spines['right'].set_color(None)
        ax.spines['left'].set_color(None)

for j, (expr_name, expr_class) in enumerate(explainers.items(), start=1):
    expr = expr_class(model, target_layers, preprocess_transform, SIZE, CH)
    for k, img in enumerate(imgs):
        exp1, _ = expr.timed_single_explain(img)
        exp2 = exp1
        while np.array_equal(exp1, exp2):
            exp2, _ = expr.timed_single_explain(img)
        for i, exp in enumerate((exp1, exp2)):
            ax = axarr[j, k * 2 + i]
            ax.imshow(exp)
            if (k * 2 + i) == 0:
                ax.set_ylabel(expr_name, fontsize=14)

            ax.xaxis.set_ticklabels([])
            ax.yaxis.set_ticklabels([])
            ax.xaxis.set_ticks([])
            ax.yaxis.set_ticks([])
            ax.spines['bottom'].set_color(None)
            ax.spines['top'].set_color(None)
            ax.spines['right'].set_color(None)
            ax.spines['left'].set_color(None)

## Imagenette accuracy calculation

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset("imagenet")

In [ ]:
model, target_layers = get_model("vgg16_bn", "imagenet")
preprocess_transform = get_preprocess_transform("imagenet")
batch_predict = get_batch_predict(model, preprocess_transform)

In [ ]:
res = []
for img, cls in tqdm(dataset):
    res.append(imagenet_idx_to_imagenette.get(np.argmax(batch_predict([img])[0])) == cls)
np.mean(res)

In [ ]:
model, target_layers = get_model("resnet50", "imagenet")
preprocess_transform = get_preprocess_transform("imagenet")
batch_predict = get_batch_predict(model, preprocess_transform)

In [ ]:
res = []
for img, cls in tqdm(dataset):
    res.append(imagenet_idx_to_imagenette.get(np.argmax(batch_predict([img])[0])) == cls)
np.mean(res)

In [ ]:
model, target_layers = get_model("densenet121", "imagenet")
preprocess_transform = get_preprocess_transform("imagenet")
batch_predict = get_batch_predict(model, preprocess_transform)

In [ ]:
res = []
for img, cls in tqdm(dataset):
    res.append(imagenet_idx_to_imagenette.get(np.argmax(batch_predict([img])[0])) == cls)
np.mean(res)

## Stability v2 comparison (using noisy images)

In [ ]:
dataset, classes, SIZE, HW, CH = init_dataset("imagenet")

In [ ]:
sample_images, sample_labels, sample_images_by_class = get_sample_images(dataset, classes, n_images_per_class=1)
imgs = sample_images
print_every_n = int(len(imgs) / 10)

In [ ]:
model, target_layers = get_model("resnet50", "imagenet")
preprocess_transform = get_preprocess_transform("imagenet")
batch_predict = get_batch_predict(model, preprocess_transform)

In [ ]:
expr = LimeExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
scores = calculate_stability_v2(imgs, exps, lambda x: expr.timed_single_explain(x)[0])
np.mean(scores)

In [ ]:
expr = ShapExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
scores = calculate_stability_v2(imgs, exps, lambda x: expr.timed_single_explain(x)[0])
np.mean(scores)

In [ ]:
expr = GradCAMExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
scores = calculate_stability_v2(imgs, exps, lambda x: expr.timed_single_explain(x)[0])
np.mean(scores)

In [ ]:
expr = IntGradExplainerAdapter(model, target_layers, preprocess_transform, SIZE, CH)

In [ ]:
exps, t = expr.timed_batch_explain(imgs)
print(f"Mean time for {len(imgs)} explanations: {t}")

In [ ]:
scores = calculate_stability_v2(imgs, exps, lambda x: expr.timed_single_explain(x)[0])
np.mean(scores)